# Best practice for plotting batch-runs

In [ ]:
import os
import bokeh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cellpy import cellreader
from cellpy.utils import ica
import holoviews
%matplotlib inline

### Setting file names and loading data

In [ ]:
my_data = cellreader.CellpyData()
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)

In [ ]:
my_data.load(filename)
my_data.set_mass(0.1)

### Checking the data

#### voltage cycles

In [ ]:
capacity, voltage = my_data.get_cap()

In [ ]:
plt.plot(capacity, voltage)

In [ ]:
cycles = my_data.get_cycle_numbers()

In [ ]:
charge_cycles = np.array([my_data.get_ccap(c) for c in cycles])

In [ ]:
for x,y in charge_cycles:
    plt.plot(x,y)

In [ ]:
def make_charge_list(data):
    minimum_v_value = np.Inf
    maximum_v_value = -np.Inf
    charge_list = []
    cycles = my_data.get_cycle_numbers()
    for cycle in cycles:
        q, v = data.get_ccap(cycle)
        d = pd.DataFrame({"q": q, "v": v})
        d.name = f"{cycle}"
        charge_list.append(d)
        v_min = v.min()
        v_max = v.max()
        if v_min < minimum_v_value:
            minimum_v_value = v_min
        if v_max > maximum_v_value:
            maximum_v_value = v_max
    return charge_list, minimum_v_value, maximum_v_value


In [ ]:
charge_dfs, minimum_v, maximum_v = make_charge_list(my_data)

In [ ]:
charge_df = pd.concat(charge_dfs, axis=1, keys=[k.name for k in charge_dfs])

In [ ]:
# discharge_cycles = np.array([my_data.get_dcap(c) for c in cycles])

### Checking ica

In [ ]:
c, v = charge_cycles[2]
dq, dv = ica.dqdv(v, c)
plt.plot(dq,dv)

In [ ]:
print(f"min, max = ({minimum_v} {maximum_v})")

## Creating dqdv-data

In [ ]:
def custom_dq_dv(capacity, voltage):
    converter = ica.Converter()
    converter.set_data(capacity, voltage)
    converter.inspect_data()
    converter.pre_process_data()
    converter.increment_data()
    converter.fixed_voltage_range = [minimum_v, maximum_v, 100]
    converter.post_process_data()
    return converter.voltage_processed, converter.incremental_capacity

In [ ]:
incremental_charge_list = []
for (cap, volt), cycle in zip(charge_cycles, cycles):
    if cap.any():
        dv, dq = custom_dq_dv(cap, volt)
        if not incremental_charge_list:
            d = pd.DataFrame({"dv": dv, f"dq": dq})
        else:
            d = pd.DataFrame({f"dq": dq})
        d.name = f"{cycle}"
        incremental_charge_list.append(d)
    else:
        print(f"{cycle} is empty")

In [ ]:
ica_df = pd.concat(incremental_charge_list, axis=1, keys=[k.name for k in incremental_charge_list])

In [ ]:
ica_df.columns.names = ["cycle", "value"]

In [ ]:
ica_df.head()

In [ ]:
ica_df.plot(x=("1","dv"))

**Success!**

### Trying Bokeh

The main benifit with using Bokeh: interactive html

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# trick to flatten the multiindex (not sure if HoloViews takes multiindex)
# ica_df.columns = ['_'.join(col).strip() for col in ica_df.columns.values]

In [ ]:
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import MultiLine, Line
from bokeh.io import show

In [ ]:
source = ColumnDataSource(ica_df)
#source.data

In [ ]:
xrange = DataRange1d()
yrange = DataRange1d()
plot = Plot(x_range=xrange, y_range=yrange, plot_width=300, plot_height=300)

In [ ]:
glyph = Line(x="1_dv", y="1_dq")

In [ ]:
plot.add_glyph(source, glyph)
show(plot)

In [ ]:
from bokeh.plotting import figure, show
p = figure(plot_width=400, plot_height=400)
p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
show(p)

In [ ]:
#ica_df.xs("dq", axis=1, level=1)

In [ ]:
p = figure()
p.line("1_dv", "1_dq", source=source)
show(p)

In [ ]:
source.column_names

In [ ]:
import itertools
from bokeh.palettes import Spectral11

line_dash_styles = [[10, 0], [20, 1], [10, 1], [5, 1]]
p = figure(name="all", plot_width=900, plot_height=400)
for col, color, line_dash in zip(source.column_names, itertools.cycle(Spectral11), itertools.cycle(line_dash_styles)):
    if col not in ["1_dv", "index"]:
        p.line("1_dv", col, source=source, color=color, legend=col.split("_")[0], line_dash=line_dash)
p.legend.location = "bottom_left"
p.legend.name = "cycle"
show(p)